### My Imports
Import Pytorch functions, as well as image loading/analysis tools

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot
from skimage.transform import rotate, AffineTransform, warp, rescale

## Create Subimages for testing

In [5]:
def rgb2ycbcr(im_rgb):
    im_rgb = im_rgb.astype(np.float32)
    im_ycrcb = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2YCR_CB) #converts the RBG colors into YCbCr colorspace
    im_ycbcr = im_ycrcb[:,:,(0,2,1)].astype(np.float32)
    im_ycbcr[:,:,0] = (im_ycbcr[:,:,0]*(235-16)+16)/255.0 #to [16/255, 235/255]
    im_ycbcr[:,:,1:] = (im_ycbcr[:,:,1:]*(240-16)+16)/255.0 #to [16/255, 240/255]
    return im_ycbcr

def imresize(im_l, s):
    if s<1:
        im_l = cv2.GaussianBlur(im_l, (7,7), 0.5)
        #im_l = cv2.resize(im_l, (0,0), fx=s, fy=s, interpolation=cv2.INTER_CUBIC)
        #s = 1/s
    im_h = cv2.resize(im_l, (0,0), fx=s, fy=s, interpolation=cv2.INTER_CUBIC)
    return im_h

    
def images_2_patches(input_dir, output_dir, patch_size, step, mag=1, crop=0, rand=False):
    #load input images
    os.makedirs(output_dir, exist_ok=True)
    files = os.listdir(input_dir)
    
    for fil in files:
        loc = os.path.join(input_dir,fil)
        #for each one, convert to ycbcr
        im = np.array(Image.open(loc))
        im = im/255.0
        
        if len(im.shape)==3 and im.shape[2]==3:
            im_ycbcr = rgb2ycbcr(im)
        else:
            im_ycbcr = np.zeros([im.shape[0], im.shape[1], 3])
            im_ycbcr[:, :, 0] = im
            im_ycbcr[:, :, 1] = im
            im_ycbcr[:, :, 2] = im
        
        im_y = im_ycbcr[:,:,0]*255
        
        im_h_y = imresize(im_y,mag)
        
        im_h_y = np.clip(im_h_y, 0, 255)
        
        #take patch_size samples of images
        h, w = im_h_y.shape
        
        cnt=0
        im_name = fil.split('.')[0]
        for i in range(0,w,step):
            for j in range(0,h,step):
                
                if i+patch_size < w:
                    #print(f'{i+patch_size} is less than {w}')
                    if j+patch_size < h:
                        #print(f'{j+patch_size} is less than {h}')
                        sample = im_h_y[j+crop:j+patch_size-crop,i+crop:i+patch_size-crop]
                        cnt+=1
                        
                        Image.fromarray(np.rint(sample).astype(np.uint8)).save(f'{output_dir}/{im_name}_{cnt}.bmp')
            
            #img_name = os.path.splitext(os.path.basename(files_gt[i]))[0]
            #Image.fromarray(np.rint(im_h).astype(np.uint8)).save('./data/{}_x{}.png'.format(img_name, UP_SCALE))
    
        #save to output folder, named <filename>_001.bmp
        

## Create Randomized Transformations

In [18]:
def create_rand(LR_input_dir, LR_output_dir, HR_input_dir, HR_output_dir):
    os.makedirs(LR_output_dir, exist_ok=True)
    os.makedirs(HR_output_dir, exist_ok=True)
    
    files = os.listdir(LR_input_dir)
    
    rot = np.random.randint(0,1, size=len(files))
    scale = 2+np.random.randint(1, size=len(files))
    trans_x = np.random.randint(0,1, size=len(files))
    trans_y = np.random.randint(0,1, size=len(files))
    

    for idx, fil in enumerate(files):
        transform = AffineTransform(translation=(trans_x[idx], trans_y[idx]))
        loc = os.path.join(LR_input_dir,fil)
        #for each one, convert to ycbcr
#        im = np.array(Image.open(loc))
        im_h_y = np.array(Image.open(loc))
        im_name = fil.split('.')[0]
        im_h_y = rotate(im_h_y,rot[idx],mode="reflect")
        im_h_y = rescale(im_h_y,scale[idx],mode="reflect", multichannel=True)
        im_h_y = warp(im_h_y,transform,mode='reflect')
        
        #Image.fromarray(np.rint(im_h_y).astype(np.uint8)).save(f'{LR_output_dir}/{im_name}_{scale[idx]}-{rot[idx]}-{trans_x[idx]}-{trans_y[idx]}.png')
        #Image.fromarray(im_h_y,'RGB').save(f'{LR_output_dir}/{im_name}_{scale[idx]}-{rot[idx]}-{trans_x[idx]}-{trans_y[idx]}.png')
        #cv2.imwrite(f'{LR_output_dir}/{im_name}_{scale[idx]}-{rot[idx]}-{trans_x[idx]}-{trans_y[idx]}.png', im_h_y)
        pyplot.imsave(f'{LR_output_dir}/{im_name}_{scale[idx]}-{rot[idx]}-{trans_x[idx]}-{trans_y[idx]}.png',im_h_y)
        
        
    for idx, fil in enumerate(files):
        transform = AffineTransform(translation=(trans_x[idx]*2, trans_y[idx]*2))
        loc = os.path.join(HR_input_dir,fil)
        #for each one, convert to ycbcr
        im_h_y = np.array(Image.open(loc))
        im_name = fil.split('.')[0]
        im_h_y = rotate(im_h_y,rot[idx],mode="reflect")
        im_h_y = rescale(im_h_y,scale[idx],mode="reflect", multichannel=True)
        im_h_y = warp(im_h_y,transform,mode='reflect')
        
        #Image.fromarray(np.rint(im_h_y).astype(np.uint8)).save(f'{HR_output_dir}/{im_name}_{scale[idx]}-{rot[idx]}-{trans_x[idx]}-{trans_y[idx]}.png') 
        pyplot.imsave(f'{HR_output_dir}/{im_name}_{scale[idx]}-{rot[idx]}-{trans_x[idx]}-{trans_y[idx]}.png',im_h_y)


In [15]:
outDir = '/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/data'
for i in range(10):
    create_rand(f'{outDir}/raw/Set5_LR_x2',f'{outDir}/shuff_Set5_LR',
                f'{outDir}/raw/Set5_HR',f'{outDir}/shuff_Set5_HR')

#create_rand('/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/data/processed_images/Set5_LR','/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/shuff_LR','/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/data/processed_images/Set5_HR','/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/shuff_HR')

(172, 114, 3)
(344, 228, 3)
(128, 128, 3)
(384, 384, 3)
(256, 256, 3)
(256, 256, 3)
(144, 144, 3)
(576, 576, 3)
(140, 140, 3)
(560, 560, 3)
(172, 114, 3)
(344, 228, 3)
(128, 128, 3)
(128, 128, 3)
(256, 256, 3)
(768, 768, 3)
(144, 144, 3)
(288, 288, 3)
(140, 140, 3)
(560, 560, 3)
(172, 114, 3)
(344, 228, 3)
(128, 128, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(144, 144, 3)
(144, 144, 3)
(140, 140, 3)
(560, 560, 3)
(172, 114, 3)
(344, 228, 3)
(128, 128, 3)
(128, 128, 3)
(256, 256, 3)
(512, 512, 3)
(144, 144, 3)
(576, 576, 3)
(140, 140, 3)
(560, 560, 3)
(172, 114, 3)
(344, 228, 3)
(128, 128, 3)
(128, 128, 3)
(256, 256, 3)
(512, 512, 3)
(144, 144, 3)
(144, 144, 3)
(140, 140, 3)
(560, 560, 3)
(172, 114, 3)
(344, 228, 3)
(128, 128, 3)
(512, 512, 3)
(256, 256, 3)
(512, 512, 3)
(144, 144, 3)
(432, 432, 3)
(140, 140, 3)
(560, 560, 3)
(172, 114, 3)
(516, 342, 3)
(128, 128, 3)
(128, 128, 3)
(256, 256, 3)
(256, 256, 3)
(144, 144, 3)
(432, 432, 3)
(140, 140, 3)
(280, 280, 3)
(172, 114, 3)
(344, 

## Cut Images Into Patches

In [11]:
def images_2_patches(input_dir, output_dir, patch_size, step, mag=1, crop=0, rand=False):
    #load input images
    os.makedirs(output_dir, exist_ok=True)
    files = os.listdir(input_dir)
    
    for fil in files:
        loc = os.path.join(input_dir,fil)
        #for each one, convert to ycbcr
        im_h_y = np.array(Image.open(loc))
        #print(im_h_y.shape)
        im_h_y = rescale(im_h_y,mag,mode="reflect", multichannel=True)
        #print(im_h_y.shape)
        #take patch_size samples of images
        h, w, c = im_h_y.shape
        
        cnt=0
        im_name = fil.split('.')[0]
        for i in range(0,w,step):
            for j in range(0,h,step):
                
                if i+patch_size < w:
                    #print(f'{i+patch_size} is less than {w}')
                    if j+patch_size < h:
                        #print(f'{j+patch_size} is less than {h}')
                        sample = im_h_y[j+crop:j+patch_size-crop,i+crop:i+patch_size-crop,:3]
                        cnt+=1
                        
                        #Image.fromarray(np.rint(sample).astype(np.uint8)).save(f'{output_dir}/{im_name}_{cnt}.bmp')
                        
                        if sample.max() > 0: #Do not save patches that are just blank
                            #print(sample.max())
                            pyplot.imsave(f'{output_dir}/{im_name}_{cnt}.png',sample)
                        else:
                            print('blank image')
            
            #img_name = os.path.splitext(os.path.basename(files_gt[i]))[0]
            #Image.fromarray(np.rint(im_h).astype(np.uint8)).save('./data/{}_x{}.png'.format(img_name, UP_SCALE))
    
        #save to output folder, named <filename>_001.bmp

In [37]:
pat = '/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/data'
#images_2_patches(f'{pat}/raw/BSD100_HR',f'{pat}/processed_images/BSD100_LR_x2',56,56, mag = 0.5, crop=0)

In [38]:
images_2_patches(f'{pat}/shuff_Set5_HR',f'{pat}/shuff_Set5_HR_patches',44, mag=1, crop=0, step =30) #50)

(840, 840, 4)
(840, 840, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(768, 768, 4)
(768, 768, 4)
(840, 840, 4)
(840, 840, 4)
(1032, 684, 4)
(1032, 684, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(1536, 1536, 4)
(1536, 1536, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(688, 456, 4)
(688, 456, 4)
(1120, 1120, 4)
(1120, 1120, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(768, 768, 4)
(768, 768, 4)
(1032, 684, 4)
(1032, 684, 4)
(1152, 1152, 4)
(1152, 1152, 4)
(1152, 1152, 4)
(1152, 1152, 4)
(280, 280, 4)
(280, 280, 4)
(256, 256, 4)
(256, 256, 4)
(1024, 1024, 4)
(1024, 1024, 4)
(280, 280, 4)
(280, 280, 4)
(1536, 1536, 4)
(1536, 1536, 4)
(1032, 684, 4)
(1032, 684, 4)
(280, 280, 4)
(280, 280, 4)
(2048, 2048, 4)
(2048, 2048, 4)
(688, 456, 4)
(688, 456, 4)
(512, 512, 4)
(512, 512, 4)
(1152, 1152, 4)
(1152, 1152, 4)
(1032, 684, 4)
(1032, 684, 4)
(1152, 1152, 4)
(1152, 1152, 4)
(288, 288, 4)
(288, 288, 4)
(1152, 1152, 4)
(1152, 1152, 4)
(288, 288, 4)
(288, 288, 4)
(576

## Create MRI images for use

In [21]:
import numpy as np
import os    # Traverse folders 
import nibabel as nib #nii Format 1 This bag will be used in general 
import imageio   # Convert to an image 
 
def nii_to_image(niifile, outpath, s=1):
    filenames = os.listdir(filepath) # Read nii Folder 
    slice_trans = []
    os.makedirs(outpath,exist_ok=True)    # New Folder
 
    for f in filenames:
    # Start reading nii Documents 
        img_path = os.path.join(filepath, f)
        img = nib.load(img_path)    # Read nii
        img_fdata = img.get_fdata()
        print(img.get_fdata().shape)
        fname = f.replace('.nii','')   # Remove nii Suffix name
 
        # Start converting to an image 
        (x,y,z) = img.shape
        for i in range(z):      #z Is a sequence of images 
            print(f'Standard deviation ={np.std(img_fdata)}')
            print(f'Mean = {np.mean(img_fdata)}')
            print(f'Min = {np.min(img_fdata)}')
            print(f'Max = {np.max(img_fdata)}')
            silce = img_fdata[:, :, i]   # You can choose which direction of slice 

            if silce.max()<=0:
                silce = np.zeros(silce.shape)
            else:
                silce = silce/silce.max()
            
            
            silce = imresize(silce, s)
            rgb_silce = np.zeros((silce.shape[0],silce.shape[1],3))
            rgb_silce[:,:,0]=silce
            rgb_silce[:,:,1]=silce
            rgb_silce[:,:,2]=silce
            rgb_silce = np.clip(rgb_silce,0,1)
            print(rgb_silce.max())
            print(rgb_silce.min())
            print(rgb_silce.shape)
            
            #imageio.imwrite(os.path.join(outpath,'{}.png'.format(i)), silce)
            pyplot.imsave(os.path.join(outpath,'{}.png'.format(i)),rgb_silce)

filepath = '/home/ross/Documents/CODE/sparse/nii'
outpath = '/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/data/raw/nii_HR_sag'
nii_to_image(filepath,outpath, s=1)


(256, 256, 204)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =52487.14133375199
Mean = 27976.323098247623
Min = 0.0
Max = 613182.6304931641
1.0
0.0
(256, 256, 3)
Standard deviation =5248

In [22]:
outDir = '/home/ross/Documents/CODE/sparse/sparse_SR/nn-code/data'
for i in range(1):
    create_rand(f'{outDir}/raw/nii_LR_sag',f'{outDir}/shuff_nii_LR_sag',
                f'{outDir}/raw/nii_HR_sag',f'{outDir}/shuff_nii_HR_sag')

In [13]:
images_2_patches(f'{outDir}/shuff_nii_sub_HR_2',f'{outDir}/shuff_nii_sub_HR_patches',44, mag=1, crop=0, step=44) #50)

blank image
blank image
blank image
blank image
blank image
blank image
blank image
blank image
blank image
blank image
blank image
blank image
